In [1]:
import chess
import math
import copy

## Evaluation of the given state
1. evaluate piece values
2. evaluate pawn position state
- 2.1. evaluate blocked pawns
- 2.2 evaluate doubled pawns
- 2.3. evaluate isolated pawns
3. evaluate mobility -open-
4. evaluate stalemate and 75 moves rule -open-

In [12]:
def static_eval(chess_board):
    chess_board = copy.deepcopy(chess_board)
    
    if chess_board.result() == "1-0":
        return math.inf
    elif chess_board.result() == "0-1":
        return -math.inf
    elif chess_board.result() == "1/2-1/2" or chess_board.is_stalemate():
        return 0
    
    value = 0
    
    
    # 1. evaluate piece values -------------------------------------------
    # Types:
    """{chess.PAWN, chess.KNIGHT, chess.BISHOP,
             chess.ROOK, chess.QUEEN, chess.KING}"""
    piece_values = [1, 
                    3, 
                    3, 
                    5,
                    9, 
                    200]
    for i, piece_value in enumerate(piece_values):
        value += len(chess_board.pieces(i+1, chess.WHITE))*piece_value
        value -= len(chess_board.pieces(i+1, chess.BLACK))*piece_value
    
    
    # 2. evaluate pawn position state ------------------------------------
    blocked_pawns_white, doubled_pawns_white, isolated_pawns_white = analyze_pawns(chess_board,chess.WHITE)
    blocked_pawns_black, doubled_pawns_black, isolated_pawns_black = analyze_pawns(chess_board,chess.BLACK)
    
    # 2.1 Blocked Pawns
    value -= 0.5 * (blocked_pawns_white - blocked_pawns_black)
    
    # 2.2 Doubled Pawns
    value -= 0.5 * (doubled_pawns_white - doubled_pawns_black)
    
    # 2.3 Isolated Pawns
    value -= 0.5 * (isolated_pawns_white - isolated_pawns_black)
    
    return value

## Calculate how many doubled pawns exist for a given file_dict

In [3]:
def count_doubled_pawns(file_dict):
    return len([file_count for file_count in file_dict if file_dict[file_count]>1])

## Calculate isolated pawns

In [4]:
def has_adjacent_pawn(file_dict,file_count):
    if file_count == 1:
        return file_dict[2]
    elif file_count == 8:
        return file_dict[7]
    else:
        return file_dict[file_count+1] or file_dict[file_count-1]

In [5]:
def count_isolated_pawns(file_dict):
    return len([file_count for file_count in file_dict if not has_adjacent_pawn(file_dict,file_count)])

## Analyze Pawns for a given color:
- Returns: tuple:
    - blocked_pawns 
    - doubled_pawns
    - isolated_pawns
    
Steps:
1. Build dict: {file_number:pawn_count} --> {1:2, 2:0, 3:1} while counting blocked pawns
2. calculate doubled pawns
3. calculate isolated pawns
4. return (blocked_pawns, doubled_pawns, isolated_pawns)

In [6]:
def analyze_pawns(chess_board,color):
    file_dict = {}
    legal_moves = set(chess_board.legal_moves)
    blocked_pawns = 0
    
    for file_count, file in enumerate(chess.FILE_NAMES,start=1):
        file_dict[file_count] = 0
        for rank in chess.RANK_NAMES:
            square = chess.parse_square(file + rank)
            if chess_board.piece_type_at(square) == chess.PAWN and board.color_at(square) == color:
                file_dict[file_count] += 1
                
                pawn_blocked = True
                for move in legal_moves:
                    if move.from_square == square:
                        pawn_blocked = False
                        legal_moves.remove(move)
                        break
                if pawn_blocked:
                    blocked_pawns+=1
    
    
    return (blocked_pawns,count_doubled_pawns(file_dict),count_isolated_pawns(file_dict))

## Main minimax function

In [10]:
def minimax(chess_board, depth, maximizing_player, moves):
    chess_board = copy.deepcopy(chess_board)
    if depth == 0 or (not chess_board.legal_moves):
        return (static_eval(chess_board), moves)

    if maximizing_player:
        max_val = -math.inf
        max_move = None
        for move in chess_board.legal_moves:
            chess_board.push(move)
            value, _ = minimax(chess_board, depth - 1, False, moves + [move])
            chess_board.pop()
            max_val = max(max_val, value)
            if max_val == value:
                max_move = move
        assert chess_board.legal_moves
        assert not max_move == None
        return max_val, moves + [max_move]
    else:
        min_val = +math.inf
        min_move = None
        for move in chess_board.legal_moves:
            chess_board.push(move)
            value, _ = minimax(chess_board, depth - 1, True, moves + [move])

            chess_board.pop()
            min_val = min(min_val, value)
            if min_val == value:
                min_move = move
        assert chess_board.legal_moves
        assert not min_move == None
        return min_val, moves + [min_move]

In [8]:
def minimax_for_color(board, color, depth):
    return minimax(copy.deepcopy(board), depth, color, [])